In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def get_box(key):
    for r_idx, r in enumerate(ROW):
        for c_idx, c in enumerate(COL):
            if (key[0] == r) and (key[-1] == c):    
                return boxes[r_idx][c_idx]

def check_cols(n, col, board):
    for key in board.keys():
        if key[-1] == col:
            if board[key] == n:
                return False
    return True

def check_rows(n, row, board):
    for key in board.keys():
        if key[0] == row:
            if board[key] == n:
                return False
    return True

def check_box(n, box):
    global box_to_vals
    if n in box_to_vals[box]:
        return False
    return True
    

In [3]:
def is_legal(n, coord, board):
    return (check_cols(n, coord[-1], board) and check_rows(n, coord[0], board) and check_box(n, get_box(coord)) and (n>0) and (n<10))

In [4]:
'''def solve(board):
    b = board.copy()
    if not 0 in b.values():
        return b
    for r in ROW:
        for c in COL:
            if b[r+c] == 0:
                for n in range(1,10):
                    if is_legal(n, r+c, b):
                        b[r+c] = n
                        result = solve(b)
                        if result != 'failure':
                            return result
                        b[r+c] = 0
                return 'failure'
                
res = solve(board)
print(res)
print(board)
                
'''

"def solve(board):\n    b = board.copy()\n    if not 0 in b.values():\n        return b\n    for r in ROW:\n        for c in COL:\n            if b[r+c] == 0:\n                for n in range(1,10):\n                    if is_legal(n, r+c, b):\n                        b[r+c] = n\n                        result = solve(b)\n                        if result != 'failure':\n                            return result\n                        b[r+c] = 0\n                return 'failure'\n                \nres = solve(board)\nprint(res)\nprint(board)\n                \n"

In [5]:
def is_complete(board):
    return (not 0 in board.values())

def all_legal_vals(coord, board):
    legal_vals = []
    for n in range(1,10):
        if is_legal(n, coord, board):
            legal_vals.append(n)
    return legal_vals

def get_mrv(open_coords, board):
    """
    Returns the unfilled box coordinate (coord) with minimum remaining [legal] values (MRV) 
    and the corresponding values (available domain)
    """
    #Could this be improved?
    possible_moves = [(all_legal_vals(coord, board), coord) for coord in open_coords]
    n_moves_per_box = [len(moves[0]) for moves in possible_moves]
    idx = np.argmin(n_moves_per_box)
    return open_coords[idx], possible_moves[idx][0]


def solve_mrv(board):
    
    b = board.copy()
    
    "Backtracking search to solve soduku"
    # If soduku is complete return it.
    if is_complete(b):
        return b
    
    # Select the MRV variable to fill
    '''
    get coordinates of available spaces to fill
    '''
    open_moves = [key for key,value in b.items() if value==0]
    
    coord, vals = get_mrv(open_moves, b)
    
    # Fill in a value and solve further (recursively), 
    # backtracking an assignment when stuck
    for value in vals:
        b[coord] = value
        result = solve_mrv(b)
        if result:
            return result
        else:
            b[coord] = 0
    return False

In [6]:
ROW = "ABCDEFGHI"
COL = "123456789"

def to_np(inp):
    bd = np.array(list(inp.values())).reshape((9,9))
    return bd

def to_dict(line):
    global ROW
    global COL
    return {ROW[r] + COL[c]: int(line[9*r+c]) for r in range(9) for c in range(9)}

sudokus = []
with open('starter/sudokus_start.txt', 'r+') as f:
    sudokus = [to_dict(i.strip()) for i in f.readlines()]

In [7]:
import time
from tqdm import tqdm

#number of boards you could solve from sudokus start.txt,
#running time statistics: min, max, mean, and standard deviation.

boxes = [[(i//3,j//3) for i in range(9)] for j in range(9)]
box_to_vals = dict()
solution_data = dict()

for idx, board in tqdm(enumerate(sudokus[:2])):
    #Reset pertinent globals:
    box_to_vals = dict()

    #board = sudokus[16]
    
    #Prep grid reference:
    for key in board.keys():
        val = board[key]
        box = get_box(key)
        if box_to_vals.get(box):
            box_to_vals[box].append(val)
        else:
            box_to_vals[box] = [val]

    start = time.time()
    solution = solve_mrv(board)
    end = time.time()

    time_to_solve = end-start

    solution_data[idx] = {'solution':solution, 'time':time_to_solve}

    if solution:
        print(to_np(solution))
    else:
        print('No solution found.')

    print(f'{(time_to_solve)} sec')


2it [00:00,  7.96it/s]

[[4 8 3 9 2 1 6 5 7]
 [9 6 7 3 4 5 8 2 1]
 [2 5 1 8 7 6 4 9 3]
 [5 4 8 1 3 2 9 7 6]
 [7 2 9 5 6 4 1 3 8]
 [1 3 6 7 9 8 2 4 5]
 [3 7 2 6 8 9 5 1 4]
 [8 1 4 2 5 3 7 6 9]
 [6 9 5 4 1 7 3 8 2]]
0.1608591079711914 sec
[[4 3 5 2 6 9 7 8 1]
 [6 8 2 5 7 1 4 9 3]
 [1 9 7 8 3 4 5 6 2]
 [8 2 6 1 9 5 3 4 7]
 [3 7 4 6 8 2 9 1 5]
 [9 5 1 7 4 3 6 2 8]
 [5 1 9 3 2 6 8 7 4]
 [2 4 8 9 5 7 1 3 6]
 [7 6 3 4 1 8 2 5 9]]
0.0867319107055664 sec


In [18]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

with open('solns.pickle', 'wb') as fp:
    pickle.dump(solution_data, fp, protocol=pickle.HIGHEST_PROTOCOL)



In [19]:
with open('solns.pickle', 'rb') as fp:
    data = pickle.load(fp)

In [23]:
times = [v['time'] for k,v in solution_data.items()]

In [33]:
box_to_vals = dict()

board = sudokus[16]

#Prep grid reference:
for key in board.keys():
    val = board[key]
    box = get_box(key)
    if box_to_vals.get(box):
        box_to_vals[box].append(val)
    else:
        box_to_vals[box] = [val]

solve_mrv(board)#to_dict('800000000003600000070090200050007000000045700000100030001000068008500010090000400'))

{'A1': 9,
 'A2': 3,
 'A3': 2,
 'A4': 5,
 'A5': 4,
 'A6': 6,
 'A7': 7,
 'A8': 8,
 'A9': 1,
 'B1': 1,
 'B2': 6,
 'B3': 4,
 'B4': 9,
 'B5': 8,
 'B6': 7,
 'B7': 3,
 'B8': 2,
 'B9': 5,
 'C1': 6,
 'C2': 7,
 'C3': 1,
 'C4': 2,
 'C5': 3,
 'C6': 5,
 'C7': 8,
 'C8': 4,
 'C9': 9,
 'D1': 8,
 'D2': 1,
 'D3': 9,
 'D4': 4,
 'D5': 6,
 'D6': 2,
 'D7': 5,
 'D8': 3,
 'D9': 7,
 'E1': 3,
 'E2': 4,
 'E3': 5,
 'E4': 7,
 'E5': 1,
 'E6': 8,
 'E7': 9,
 'E8': 6,
 'E9': 2,
 'F1': 7,
 'F2': 2,
 'F3': 8,
 'F4': 1,
 'F5': 9,
 'F6': 3,
 'F7': 4,
 'F8': 5,
 'F9': 6,
 'G1': 4,
 'G2': 5,
 'G3': 6,
 'G4': 8,
 'G5': 7,
 'G6': 1,
 'G7': 2,
 'G8': 9,
 'G9': 3,
 'H1': 2,
 'H2': 9,
 'H3': 3,
 'H4': 6,
 'H5': 5,
 'H6': 4,
 'H7': 1,
 'H8': 7,
 'H9': 8,
 'I1': 5,
 'I2': 8,
 'I3': 7,
 'I4': 3,
 'I5': 2,
 'I6': 9,
 'I7': 6,
 'I8': 1,
 'I9': 4}